[Home](../../README.md)

### Data Preprocessing

This is a demonstration of data wrangling using [Pandas](https://pandas.pydata.org/) the library for data analysis and manipulation.

This Jupyter Notepad demonstrates different processes you can apply to your data before data wrangling. For this demonstration we wrangle the diabetes data set you previewed in the last Jupyter Notebook.

> [!Note]
> None of these processes are destructive to the source CSV.

#### Load the required dependencies

In [92]:
# Import frameworks
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

####  Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [93]:
data_frame = pd.read_csv("2.1.1.diabeties_sample_data.csv")

#### Dealing with null values

Null values during data analysis can cause runtime errors and unexpected results. It is important to identify null values and deal with them appropriately before training a model.

The `isnull().sum()` method call returns the null values in any column.

In [65]:
data_frame.isnull().sum()

AGE                  1
SEX                  1
BMI                  0
BP                   0
Total Cholesterol    0
Blood Sugar Level    0
Target               0
dtype: int64

If you have null data there are many ways to deal with the empty/null values. These are the two most common approaches.
1. Remove any row with a null value with a `dropna()` method call.
2. Replace missing values with another value with a `fillna()` method call. Generally, we use mean value for numerical columns because it may cause minimal changes in your mathematical analysis while maintaining the original size of the data.

Students should reflect why this example removes the null 'SEX' but replacing the mean 'AGE'?

In [66]:
# Remove Null values
data_frame = data_frame.dropna(subset=['SEX'])
data_frame.isnull().sum()

AGE                  1
SEX                  0
BMI                  0
BP                   0
Total Cholesterol    0
Blood Sugar Level    0
Target               0
dtype: int64

In [67]:
# Replace Null values with the mean value for the column
data_frame['AGE'] = data_frame['AGE'].fillna(data_frame['AGE'].mean())
data_frame.isnull().sum()

AGE                  0
SEX                  0
BMI                  0
BP                   0
Total Cholesterol    0
Blood Sugar Level    0
Target               0
dtype: int64

#### Remove Duplicates

Duplicate data can have detrimental effects on your machine learning models and outcomes, such as reducing data diversity and representativeness, which can lead to overfitting or biased models.

The `duplicated().sum()` method call returns the count of duplicate rows in the data frame.

In [68]:
data_frame.duplicated().sum()

np.int64(1)

The `drop_duplicates()` method call can be then stored back onto the data_frame variable removing the duplicates.

In [69]:
data_frame = data_frame.drop_duplicates()
data_frame.duplicated().sum()

np.int64(0)

#### Replace data

We can run a lambda function on a column to modify its values. For a simple example, let’s convert the Sex to lowercase. To run a function over a complete column, we can use the apply method which iterates over each row and modifies the values.

In [70]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda x: x.lower())
data_frame['SEX'].head()

0    female
1      male
2    female
3      male
4      male
Name: SEX, dtype: object

We can check that there are no data entry errors by the `unique()` method call.

In [71]:
data_frame['SEX'].unique()

array(['female', 'male', 'girl'], dtype=object)

In [72]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: 'male' if gender.lower() == 'male' else 'female')
data_frame['SEX'].unique()

array(['female', 'male'], dtype=object)

#### Encoding categorical variables

Data Encoding converts textual data into numerical format, so that it can be used as input for algorithms to process. The reason for encoding is that most machine learning algorithms work with numbers and not with text or categorical variables.

To encode the 'SEX' column you will assigning a number value to the gender. Because the data set only provides 2 values we will use -1 and 1.

In [73]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: -1 if gender.lower() == 'male' else 1 if gender.lower() == 'female' else None)
print(data_frame['SEX'].head())

0    1
1   -1
2    1
3   -1
4   -1
Name: SEX, dtype: int64


#### Remove outliers

Outliers can skew your analysis on numerical columns, and it is important to remove them. We can use the 25th and 75th quartile on numerical data, to get the inter-quartile range. This allows us to estimate an acceptable range, and we can then filter out any values outside this range. Mathematically, outliers are values occurring outside 1.5 times the interquartile range (IQR) from the first quartile (Q1) or third quartile (Q3).

In [87]:
#get the inter-quartile range on the salary column
print(data_frame['BP'].describe())
Q1 = data_frame['BP'].quantile(0.25)
Q3 = data_frame['BP'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a BP above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')


count    443.000000
mean      94.661354
std       14.219157
min       51.000000
25%       84.000000
50%       93.000000
75%      105.000000
max      141.000000
Name: BP, dtype: float64
Outliers are a BP above 136.5 or below 52.5


In [88]:
# Filter salaries within the acceptable range
data_frame = data_frame[(data_frame['BP'] >= Q1 - 1.5 * IQR) & (data_frame['BP'] <= Q3 + 1.5 * IQR)]
print(data_frame['BP'].describe())

count    440.000000
mean      94.556773
std       13.785608
min       62.000000
25%       84.000000
50%       93.000000
75%      105.000000
max      133.000000
Name: BP, dtype: float64


#### Scaling features to a common range

Scaling the features makes it easier for machine learning algorithms to find the optimal solution, as the different scales of the features do not influence them.

In [96]:
scaler = MinMaxScaler()
data_frame['BP'] = scaler.fit_transform(data_frame['BP'].values.reshape(-1,1))
print(data_frame['BP'].describe())

count    443.000000
mean       0.485126
std        0.157991
min        0.000000
25%        0.366667
50%        0.466667
75%        0.600000
max        1.000000
Name: BP, dtype: float64


#### Save the wrangled data to CSV

In [89]:
data_frame.to_csv('../2.2.Feature_Engineering/2.2.1.preprocessed_data.csv')